In [28]:
def parse_input_file(filename):
    with open(filename, "r") as file:
        lines = file.readlines()
    
    # Parse first line (D, R, T)
    D, R, T = map(int, lines[0].split())
    
    # Parse resources
    resources = []
    for i in range(1, R + 1):
        parts = lines[i].split()
        resource = {
            "Resource ID": int(parts[0]), 
            "Activation Cost": int(parts[1]),  # One-time initial expenditure
            "Periodic Cost": int(parts[2]),    # Recurring maintenance cost per turn
            "Active Turns": int(parts[3]),     # Turns the resource stays active
            "Downtime Turns": int(parts[4]),   # Turns needed for maintenance after a cycle
            "Life Cycle": int(parts[5]),       # Total lifespan of the resource
            "Buildings Powered": int(parts[6]),# Number of buildings it supports per active turn
            "Special Effect": parts[7],        # Unique effect or property
            "Efficiency Rating": int(parts[8]) if len(parts) > 8 else None  # Additional performance metric
        }
        resources.append(resource)
    
    # Parse turns
    turns = []
    for i in range(R + 1, R + 1 + T):
        TMt, TXt, TRt = map(int, lines[i].split())
        turns.append({"Minimum Buildings": TMt, "Maximum Buildings": TXt, "Profit": TRt})
    
    return D, R, T, resources, turns

level = "5-carson"

D, R, T, resources, turns = parse_input_file(f"../inputs/{level}.txt")

# Print parsed data
print("Initial Capital:", D)
print("Total Resources:", R)
print("Game Turns:", T)
print("Resources:", resources)
print("Turns:", turns)



Initial Capital: 1000
Total Resources: 30
Game Turns: 1000
Resources: [{'Resource ID': 0, 'Activation Cost': 185, 'Periodic Cost': 53, 'Active Turns': 7, 'Downtime Turns': 3, 'Life Cycle': 13, 'Buildings Powered': 3, 'Special Effect': 'X', 'Efficiency Rating': None}, {'Resource ID': 1, 'Activation Cost': 267, 'Periodic Cost': 22, 'Active Turns': 4, 'Downtime Turns': 0, 'Life Cycle': 11, 'Buildings Powered': 1, 'Special Effect': 'B', 'Efficiency Rating': 1}, {'Resource ID': 2, 'Activation Cost': 186, 'Periodic Cost': 17, 'Active Turns': 10, 'Downtime Turns': 1, 'Life Cycle': 19, 'Buildings Powered': 1, 'Special Effect': 'X', 'Efficiency Rating': None}, {'Resource ID': 3, 'Activation Cost': 215, 'Periodic Cost': 36, 'Active Turns': 9, 'Downtime Turns': 2, 'Life Cycle': 30, 'Buildings Powered': 2, 'Special Effect': 'X', 'Efficiency Rating': None}, {'Resource ID': 4, 'Activation Cost': 217, 'Periodic Cost': 35, 'Active Turns': 2, 'Downtime Turns': 6, 'Life Cycle': 27, 'Buildings Powered': 

In [27]:

class Resource:
    current_active_turns = 0
    current_maintenance_turns = 0
    remaining_lyfecycles = 0
    # not in maintenance
    active = False
    # permanently dead
    dead = False
    game = None
    def __init__(self,  **kwargs):
        self.resource_id = kwargs["Resource ID"]
        self.activation_cost = kwargs["Activation Cost"]
        self.periodic_cost = kwargs["Periodic Cost"]
        self.active_turns = kwargs["Active Turns"]
        self.downtime_turns = kwargs["Downtime Turns"]
        self.life_cycle = kwargs["Life Cycle"]
        self.buildings_powered = kwargs["Buildings Powered"]
        self.special_effect = kwargs["Special Effect"]
        self.efficiency_rating = kwargs["Efficiency Rating"]

        self.current_active_turns = self.active_turns
        self.remaining_lyfecycles = self.life_cycle
    def update(self):
        if self.current_active_turns > 0:
            self.current_active_turns -= 1
            if self.current_active_turns == 0 and self.active:
                self.active = False
                self.current_maintenance_turns = self.downtime_turns
        elif self.current_maintenance_turns > 0:
            self.current_maintenance_turns -= 1
            if self.current_maintenance_turns == 0 and not self.active:
                if self.remaining_lyfecycles > 0:
                    self.active = True
                    self.current_active_turns = self.active_turns
                    self.remaining_lyfecycles -= 1
                else:
                    self.dead = True

def calculate_new_buildings_powered(resource, game):
    base_power = resource.buildings_powered
    total_additional_effect = 0

    # Iterate through all active resources in the game
    for active_resource in game.current_resources:
        # If the resource is of type A and is active, add its efficiency rating to the total effect
        if active_resource.special_effect == "A" and active_resource.active:
            total_additional_effect += active_resource.efficiency_rating
    
    # Calculate the new buildings powered value by adding the total effect from active A resources
    if total_additional_effect > 0:
        new_buildings_powered = base_power * (1 + total_additional_effect / 100)
    else:
        new_buildings_powered = base_power * (1 - abs(total_additional_effect) / 100)

    # Floor the final value to an integer as per the rule
    return int(new_buildings_powered)

def calculate_new_thresholds(current_turn, game):
    # Get the current turn's thresholds (TM and TX) from the game state
    tm = current_turn["Minimum Buildings"]
    tx = current_turn["Maximum Buildings"]
    
    total_additional_effect = 0  # To accumulate the effects of active B-type resources
    
    # Apply the effect of each active Resource B on the thresholds
    for active_resource in game.current_resources:
        if active_resource.special_effect == "B" and active_resource.active:
            efficiency = active_resource.efficiency_rating
            total_additional_effect += efficiency  # Add the efficiency of active B resources
            
    # Apply the accumulated effect to the thresholds
    if total_additional_effect > 0:
        # Green Resource (increase thresholds)
        tm = int(tm * (1 + total_additional_effect / 100))  # Increase minimum threshold by percentage
        tx = int(tx * (1 + total_additional_effect / 100))  # Increase maximum threshold by percentage
    else:
        # Non-Green Resource (decrease thresholds)
        tm = int(tm * (1 - abs(total_additional_effect) / 100))  # Decrease minimum threshold by percentage
        tx = int(tx * (1 - abs(total_additional_effect) / 100))  # Decrease maximum threshold by percentage

    # Ensure that TM and TX do not go below 0
    tm = max(0, tm)
    tx = max(0, tx)
    
    return tm, tx
def calculate_new_profit(current_turn, game):
    profit = current_turn["Profit"]
    # Calculate the new profit of the current turn based on the effects of active resources of type D
    total_additional_effect = 0
    for active_resource in game.current_resources:
        if active_resource.special_effect == "D" and active_resource.active:
            total_additional_effect += active_resource.efficiency_rating
    if total_additional_effect > 0:
        profit = profit * (1 + total_additional_effect / 100)
    else:
        profit = profit * (1 - abs(total_additional_effect) / 100)
    return profit
class Game:
    def __init__(self, D, R, T, resources, turns):
        self.D = D
        self.R = R
        self.T = T
        self.available_resources = [Resource(**resource) for resource in resources]
        self.turns = turns
        self.current_turn_id = 0
        self.current_budget = D
        self.current_resources = []
    
    def get_currently_active_buildings(self):
        total_buildings_powered = 0

        # For each active resource, calculate the updated buildings powered based on active "A" resources
        for resource in self.current_resources:
            if resource.active:
                # Apply the effect of any active "A" resources to this resource's buildings powered
                updated_buildings_powered = calculate_new_buildings_powered(resource, self)
                total_buildings_powered += updated_buildings_powered
        
        return total_buildings_powered

    def perform_turn(self, bought_resources_list: list):
        # periodic costs
        for resource in self.current_resources:
            if resource.dead:
                self.current_resources.remove(resource)
                continue
            self.current_budget -= resource.periodic_cost
        
        # update resources + activation costs
        for resource in bought_resources_list:
            self.current_budget -= resource.activation_cost
            resource.active = True
            resource.current_active_turns = resource.active_turns
            self.current_resources.append(resource)
        
        # add profits
        current_turn = self.turns[self.current_turn_id]
        # Calculate the new thresholds based on the current turn and the game state
        max_buildings, min_buildings = calculate_new_thresholds(current_turn, self)
        profit = calculate_new_profit(current_turn, self)
        if self.get_currently_active_buildings() >= min_buildings:
            self.current_budget += profit * min(max_buildings, self.get_currently_active_buildings())
        
        for resource in self.current_resources:
            resource.update()
        return profit * min(max_buildings, self.get_currently_active_buildings())

In [31]:
import random
# Assuming available resources are provided in the form of dictionaries with the correct structure for each resource.
available_resources = resources.copy()  # This is a list of dictionaries with resource data
available_budget = D
active_resources = []  # List of currently active resources (resource instances)
game = Game(D, R, T, available_resources, turns)  # Initialize the game
profits = []
# Game sequence for T turns
for turn in range(T):
    
    resources_to_activate = []  # To store the resources to activate in this turn
    total_buildings_powered = sum([resource.buildings_powered for resource in active_resources if resource.active])  # Get total buildings powered by active resources

    # generate a random resource to buy between A, B, C, D, E , X
    resource_types = ["A", "B", "C", "D", "X"]
    
    # Loop to find a valid resource with an Efficiency Rating > 0
    while True:
        # Randomly pick a resource type
        random_resource = 'A'
        
        # Find a valid resource matching the criteria
        resource_0_data = next((r for r in available_resources if r['Special Effect'] == random_resource and r['Efficiency Rating'] > 0), None)
        
        # If we found a valid resource, break the loop
        if resource_0_data:
            break
    # Check if Resource 0 exists
    if resource_0_data:
        # Convert the resource data into the corresponding resource object
        resource_0 = Resource(**resource_0_data)  # create_resource will create an instance of the correct class
        
        # Calculate how many times we can afford to buy Resource 0 this turn
        max_affordable = available_budget // resource_0.activation_cost
        
        # Calculate how many we can buy without exceeding the max allowed buildings (based on current turn limits)
        if resource_0.buildings_powered > 0:

            max_allowed_by_buildings = (turns[turn]["Maximum Buildings"] - total_buildings_powered) // resource_0.buildings_powered
        else :
            max_allowed_by_buildings = 0
        # Final number of Resource 0 to buy: we can buy either the maximum affordable or the maximum allowed by building capacity
        num_to_buy = min(max_affordable, max_allowed_by_buildings)
        
        # Add this resource object to the list of resources to activate in this turn
        resources_to_activate = [resource_0] * num_to_buy

    # Perform the turn with the resources to be activated (as resource objects)
    profit_per_turn = game.perform_turn(resources_to_activate)
    profits.append(profit_per_turn)
    # Print the output in the required format
    print(f"{turn} {len(resources_to_activate)}", " ".join(str(resource.resource_id) for resource in resources_to_activate))
print(sum(profits))

0 2 28 28
1 2 28 28
2 2 28 28
3 2 28 28
4 2 28 28
5 2 28 28
6 2 28 28
7 2 28 28
8 2 28 28
9 2 28 28
10 2 28 28
11 2 28 28
12 2 28 28
13 2 28 28
14 2 28 28
15 2 28 28
16 2 28 28
17 2 28 28
18 2 28 28
19 2 28 28
20 2 28 28
21 2 28 28
22 2 28 28
23 2 28 28
24 2 28 28
25 2 28 28
26 2 28 28
27 2 28 28
28 2 28 28
29 2 28 28
30 2 28 28
31 2 28 28
32 2 28 28
33 2 28 28
34 2 28 28
35 2 28 28
36 2 28 28
37 2 28 28
38 2 28 28
39 2 28 28
40 2 28 28
41 2 28 28
42 2 28 28
43 2 28 28
44 2 28 28
45 2 28 28
46 2 28 28
47 2 28 28
48 2 28 28
49 2 28 28
50 2 28 28
51 2 28 28
52 2 28 28
53 2 28 28
54 2 28 28
55 2 28 28
56 2 28 28
57 2 28 28
58 2 28 28
59 2 28 28
60 2 28 28
61 2 28 28
62 2 28 28
63 2 28 28
64 2 28 28
65 2 28 28
66 2 28 28
67 2 28 28
68 2 28 28
69 2 28 28
70 2 28 28
71 2 28 28
72 2 28 28
73 2 28 28
74 2 28 28
75 2 28 28
76 2 28 28
77 2 28 28
78 2 28 28
79 2 28 28
80 2 28 28
81 2 28 28
82 2 28 28
83 2 28 28
84 2 28 28
85 2 28 28
86 2 28 28
87 2 28 28
88 2 28 28
89 2 28 28
90 2 28 28
91 2 28 2